## Prueba de modelos ##

### Version 4: Modelo con 2 DER, planificación anual, granularidad horaria ###

Si el modelo tiene que optimizar el balance por horas, tiene 8640 VD, pero sigue teniendo 2 VD binarias

In [1]:
%load_ext autoreload

In [2]:
%autoreload 2

In [3]:
import pandapower as pp

In [4]:
import os

In [5]:
import pandas as pd

In [6]:
import pyomo.environ as pe

In [7]:
import math

In [8]:
import random

In [9]:
import itertools

In [10]:
import matplotlib.pyplot as plt

In [11]:
import numpy as np

In [12]:
import ModelWriters.SimpleBusbar

Variables que define el modelo:

y para el año

d para el día del año

h para la hora del día

dt la granularidad del modelo, en horas


temp la temperatura

wv la velocidad del viento

I la irradiación solar


eg el crecimiento económico en pu respecto al año 0


Estas variables llegan como parámetros en forma de diccionario en el argumento 'model_status' (ver este nombre)

Todas las funciones reciben este argumento, la lógica de la función indica que valor retorna. Pr ejemplo, si la granularidad es 24 h, debe retornar el valor medio del parámetro producido.

## Construcción de Escenarios

In [32]:
import ModelWriters.Simulation.Solar as Solar
import ModelWriters.Simulation.Demand as Demand

In [33]:
dias = 3
años = 5

In [34]:
#el rango de horas:
T_i = range(24)

#el rango de dias:
Dias_i = range(dias)

Escenarios_i = range(len(T_i)*len(Dias_i))

In [35]:
m_s_base = [{'y': 0, 'd': 0, 'dd':1.0, 'h': 0.0, 'dt': 1.0, 'temp': 20.0, 'I':1000.0, 'wv': 10.0, 'eg': 1.0}]

registros = len(Dias_i)*len(T_i)

Escenarios =  pd.DataFrame(m_s_base, index = Escenarios_i)

reg = 0
for d in Dias_i:
    for h in T_i:
        Escenarios['d'][reg] = d
        Escenarios['h'][reg] = h
        Escenarios['I'][reg] = Solar.solar_irradiance_seasoned_randomized(d, h)
        Escenarios['wv'][reg] = random.uniform(1.0, 20.0)
        reg += 1

In [36]:
Escenarios

,y,d,dd,h,dt,temp,I,wv,eg
0,0,0,1.0,0.0,1.0,20.0,0.000000,9.065020,1.0
1,0,0,1.0,1.0,1.0,20.0,0.000000,12.948819,1.0
2,0,0,1.0,2.0,1.0,20.0,0.000000,13.296281,1.0
3,0,0,1.0,3.0,1.0,20.0,0.000000,7.174769,1.0
4,0,0,1.0,4.0,1.0,20.0,0.000000,3.189514,1.0
...,...,...,...,...,...,...,...,...,...
67,0,2,1.0,19.0,1.0,20.0,202.282138,7.651756,1.0
68,0,2,1.0,20.0,1.0,20.0,15.960435,18.469335,1.0
69,0,2,1.0,21.0,1.0,20.0,0.000000,4.107919,1.0
70,0,2,1.0,22.0,1.0,20.0,0.000000,13.618208,1.0


## Model Tests

In [13]:
from ModelWriters.Resources.ExtGrid import ExtGrid

In [14]:
from ModelWriters.Resources.Load import Load

In [15]:
from ModelWriters.Resources.BaseGen import Generator

In [66]:
from ModelWriters.Resources.Storage import Storage

In [67]:
model = pe.ConcreteModel()

In [21]:
exg = ExtGrid('EXT')

In [22]:
bg = Generator('GEN')

In [23]:
load = Load('LOAD')

In [68]:
storage = Storage('STORAGE')

In [25]:
exg['pr_mw'] = 5.0
exg['ic_0_mu'] = 1.2
exg['ic_1_mu'] = 0.2
exg['oc_0_mu'] = 0.001
exg['oc_1_mu'] = 0.005

#exg.pa_pu = wind_output

In [37]:
load['pr_mw'] = .5
load['ic_0_mu'] = 1.2
load['ic_1_mu'] = 0.2
load['oc_0_mu'] = 0.001
load['oc_1_mu'] = 0.005
load['pa_pu'] = Demand.scalonated_seasoned_randomized_demand

#exg.pa_pu = wind_output

In [38]:
exg.initialize_model(model, Escenarios)

In [39]:
bg['pr_mw'] = 5.0
bg['ic_0_mu'] = 1.2
bg['ic_1_mu'] = 0.2
bg['oc_0_mu'] = 0.001
bg['oc_1_mu'] = 0.005


In [40]:
bg.initialize_model(model, Escenarios)

In [41]:
load.initialize_model(model, Escenarios)

In [42]:
load.available_power(10)

0.2623666302630328

In [43]:
bg.available_power(15)

In [44]:
print(bg.initial_cost())

1.2*GEN_create + 0.2*GEN_pr_mw


In [45]:
print(load.initial_cost())

1.3


In [46]:
print(bg.operating_cost(10))

0.001 + 0.005*GEN_p_mw[10]


In [47]:
print(load.operating_cost(10))

0.001 + 0.005*LOAD_p_mw[10]


In [48]:
print(bg.active_power(10))

GEN_p_mw[10]


In [49]:
print(load.active_power(10))

-0.2633838307853684


In [69]:
storage.initialize_model(model, Escenarios)

In [70]:
model.pprint()

7 Set Declarations
    STORAGE_p_constraint_charge_index : Size=1, Index=None, Ordered=False
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :   72 : {0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71}
    STORAGE_p_constraint_pr_index : Size=1, Index=None, Ordered=False
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :   72 : {0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71}
    STORAGE_p_constraint_soc_index : Size=1, Index=None, Ordered=False
        Key  : Dimen : Domain : Size

In [72]:
hasattr(Escenarios, 'dt')

True